### ROT: Detecting and Forecasting the Occlusion Events between the Sun and Clouds in Realtime - Cont.
Focus: Measuring the accuracy of the forecasts vs the SSD detection ground truths.

In [ ]:
import os
import queue
import threading
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
# This is needed to display the images.
%matplotlib inline

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
from rot_helper.thread_functions import sky_images_generator
from rot_helper.ssd_detection import detection_data, post_process_detection_data

from rot_helper.lstm_forecast import forecast_inferrence, return_sequence_ssd_detection_data
from rot_helper.lstm_forecast import n_windowed_forecast_inferrence, forecasted_occlusion_position
from rot_helper.lstm_forecast import forecast_report_on_occlusions_predictions_area
from rot_helper.lstm_forecast import longterm_forecast_report_on_occlusions_predictions_area
from rot_helper.lstm_forecast import post_process_ssd_detection_data_for_forecasting
from rot_helper.lstm_forecast import one_forecast_accuracy, multi_forecast_accuracy

In [ ]:
DEBUG=True

In [ ]:
forecast_accuracy_dict = {}

#### Sky Camera Files for Images Captured on the <2017_07_15> Day.

In [ ]:
sky_camera_images_dir = '../experiments/experiment1/2017_07_15'
sky_camera_images_files = os.listdir(sky_camera_images_dir)
sky_camera_images_files.sort() # Sorted
sky_camera_images_files[:5]

In [ ]:
len(sky_camera_images_files)

In [ ]:
if DEBUG:
    # Extract a sequence of images that are part of the test part of the forecast model dataset. 
    # Train: (7024, 50, 30)
    # Test: (3030, 50, 30)
    # Forecast_Test_Range [-3030: -2030] ~ about 1000 sample images and their sequences.
    sky_camera_images_files = sky_camera_images_files[-3130: -2030]

In [ ]:
len(sky_camera_images_files)

In [ ]:
sky_camera_images_files[0]

#### Trained ML models hyper parameters

In [ ]:
seq_num             = 50
num_bboxes          = 5
max_bboxes_vec_size = 6 * num_bboxes # About 6 points data for each of the 5 bboxes per an image.

In [ ]:
forecasting_model_name = '../exported_ml_models/lstm_forecasting_model/toesc_forecasting_keras_lambda_model.h5' # A simple forecasting model to predict the next bboxes

#### Data Structures to hold intermediary data 

In [ ]:
# To capture inbound images from the ground sky camera in realtime (i.e., a new image per second).
queue_of_inbound_sky_images = queue.Queue()

In [ ]:
# To capture a sequence of images of length = seq_num (i.e., 50) to be used to forecast the next image detection data.
queue_of_images_sequence_for_forecast_task = queue.Queue(maxsize=seq_num)  

In [ ]:
# To save in memory the current ssd detection data
ssd_detection_data_dict = {}

#### Spawn the producer thread

In [ ]:
queue_of_inbound_sky_images.qsize() # Test<1>

In [ ]:
t = threading.Thread(target=sky_images_generator, args=(sky_camera_images_files, queue_of_inbound_sky_images,))
t.start()

In [ ]:
queue_of_inbound_sky_images.qsize() # Test<2,3, ..., k>

#### Get an image file from the queue in a FIFO format

In [ ]:
sample_image_filename = queue_of_inbound_sky_images.get()

In [ ]:
sample_image_filename

### Sample Image Detection Data

#### Env setup

In [ ]:
sample_image_file = os.path.join(sky_camera_images_dir, sample_image_filename)

In [ ]:
results_detection_data = detection_data(sample_image_file)

In [ ]:
results_detection_data

In [ ]:
ssd_detection_data = post_process_detection_data(results_detection_data)

In [ ]:
ssd_detection_data_dict[list(ssd_detection_data.keys())[0]] = list(ssd_detection_data.values())[0]

In [ ]:
ssd_detection_data_dict

### Simulate inbound sky images

- Run SSD detection
- Create sequence of sky images to run forecast/predictions of future frame detection data and visualize them
- Run 24/7 or until stopped

In [ ]:
# Initial our dictionary for keeping track of the forecast accuracy
forecast_windows = [1, 5, 10, 25, 50, 75, 100]

for i in forecast_windows:
    forecast_accuracy_dict[str(i)] = []

In [ ]:
sim_counter = 0

In [ ]:
while(1):
    get_the_next_sky_image_filename = queue_of_inbound_sky_images.get() # FIFO mode
    print("\n:: Current sky image name: ", get_the_next_sky_image_filename)
    
    current_sky_image_filepath = os.path.join(sky_camera_images_dir, get_the_next_sky_image_filename)
    
    # Run SSD Detection
    results_detection_data = detection_data(current_sky_image_filepath)
    ssd_detection_data = post_process_detection_data(results_detection_data)
    
    # Append it into the dict of ssd detection data
    ssd_detection_data_dict[list(ssd_detection_data.keys())[0]] = list(ssd_detection_data.values())[0]
    candidate_key = list(ssd_detection_data.keys())[0]
    
    #(0)
    if queue_of_images_sequence_for_forecast_task.qsize() < seq_num:
        queue_of_images_sequence_for_forecast_task.put(candidate_key)
    
    #(1) If the forecast queue reaches seqNum, run the forecast model for the next 5, 10, 20, 40, 60, 80, 100 frames
    # And plot the frequency of predicted number of occlusion occurences in those predicted n-frames.
    if queue_of_images_sequence_for_forecast_task.qsize() == seq_num:
        print("\n\nTime to begin the forecast ...")
        
        # Pop the oldest entry from the queue and use it to extract seq_num detection data from the 
        # ssd_detection_data_dict
        popped_sky_image_filename = queue_of_images_sequence_for_forecast_task.get()
        print("Popped: ", popped_sky_image_filename)
        
        # Create the sequence of detection data to be used for the prediction/forecast task
        extracted_seq_num_detection_data = return_sequence_ssd_detection_data(popped_sky_image_filename, ssd_detection_data_dict, seq_num, max_bboxes_vec_size)
        #print("\nExtracted: ", extracted_seq_num_detection_data)
        print("\nExtracted data hidden.\n")
        
        # Next 1st Frame Forecast
        next_frame_forecast = forecast_inferrence(extracted_seq_num_detection_data, forecasting_model_name)
        # Later compare it with the candidtate_key (visually)
        print(":: Next 1st Frame Forecast: ", next_frame_forecast, "\n")
        
        num_occlusions_bool = forecast_report_on_occlusions_predictions_area(next_frame_forecast)
        print("\n:: Next 1st Frame Forecast -- Summary Report:\n\t >> In regards to the presence of an occlusion event in the next Frame, the forecast says: ", num_occlusions_bool, "\n")
        
        forecast_accuracy = one_forecast_accuracy(candidate_key, num_occlusions_bool)
        print("\n:: forecast_accuracy: ", forecast_accuracy)
        forecast_accuracy_dict["1"].append(forecast_accuracy)
        
        ### Generic Forecast model for n-horizon window: Next [5, 10, 25, 50, 75, 100] Frames Forecast
        forecast_window = [5, 10, 25, 50, 75, 100]

        for i in forecast_window:
            print("\n\n:: Auto forecast the next <", i, "> frames.\n")
            next_n_frames_forecasts = n_windowed_forecast_inferrence(i, extracted_seq_num_detection_data, forecasting_model_name)

            # Forecast Report
            #print("\n:: Summary of the ", i, " forecasts:\n", next_n_frames_forecasts, "\n")
            print("\n:: Summary of the ", i, " forecasts data is hidden.\n")

            occlusion_bool_list = longterm_forecast_report_on_occlusions_predictions_area(next_n_frames_forecasts)
            print("\n:: Next", i, "Frames Forecast -- Summary Report:\n\t >> In regards to the presence of an occlusion event in the next Frames, the forecast says: ", occlusion_bool_list, "\n")
            print(":: Among the ", i, "Frames Forecasts, the Occlusion events appear in these next images: ", forecasted_occlusion_position(occlusion_bool_list))

            n_forecast_accuracy = multi_forecast_accuracy(candidate_key, occlusion_bool_list, i, sky_camera_images_files, sky_camera_images_dir)
            forecast_accuracy_dict[str(i)].append(n_forecast_accuracy)
            print("\n:: ", i, "th forecast_accuracy: ", n_forecast_accuracy)
        
        sim_counter += 1
        
        if sim_counter == 1000:
            break # Only collecting up to 1000 to evaluate the forecast accuracy.
        
        #(2) Put new entry in the forecast queue (0-1)
        queue_of_images_sequence_for_forecast_task.put(candidate_key)
        print("\n\n:: Newly Put: \n\t", candidate_key)

### Forecast accuracy evaluation at different forecast time window [1, 5, ..., 100]

In [ ]:
forecast_windows = [1, 5, 10, 25, 50, 75, 100]

In [ ]:
summary_accuracy_dict = {}

In [ ]:
for i in forecast_windows:
    avg_accuracy = sum(forecast_accuracy_dict[str(i)])/len(forecast_accuracy_dict[str(i)])
    summary_accuracy_dict[i] = avg_accuracy

In [ ]:
summary_accuracy_dict

In [ ]:
plt.plot(list(summary_accuracy_dict.keys()), list(summary_accuracy_dict.values()))